In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
_FOLDER = 'drive/MyDrive/A2_Data/T3_Data/images/'
_TRAIN_ANGRY_FOLDER = _FOLDER + 'train/angry'
_TRAIN_DISGUST_FOLDER = _FOLDER + 'train/disgust'
_TRAIN_FEAR_FOLDER = _FOLDER + 'train/fear'
_TRAIN_HAPPY_FOLDER = _FOLDER + 'train/happy'
_TRAIN_NEUTRAL_FOLDER = _FOLDER + 'train/neutral'
_TRAIN_SAD_FOLDER = _FOLDER + 'train/sad'
_TRAIN_SURPRISE_FOLDER = _FOLDER + 'train/surprise'

_VALIDATION_ANGRY_FOLDER = _FOLDER + 'validation/angry'
_VALIDATION_DISGUST_FOLDER = _FOLDER + 'validation/disgust'
_VALIDATION_FEAR_FOLDER = _FOLDER + 'validation/fear'
_VALIDATION_HAPPY_FOLDER = _FOLDER + 'validation/happy'
_VALIDATION_NEUTRAL_FOLDER = _FOLDER + 'validation/neutral'
_VALIDATION_SAD_FOLDER = _FOLDER + 'validation/sad'
_VALIDATION_SURPRISE_FOLDER = _FOLDER + 'validation/surprise'

_TRAINING_DIRECTORIES_LABELS = [
  (_TRAIN_ANGRY_FOLDER,'angry'),
  (_TRAIN_DISGUST_FOLDER,'disgust'),
  (_TRAIN_FEAR_FOLDER,'fear'),
  (_TRAIN_HAPPY_FOLDER,'happy'),
  (_TRAIN_NEUTRAL_FOLDER,'neutral'),
  (_TRAIN_SAD_FOLDER,'sad'),
  (_TRAIN_SURPRISE_FOLDER,'surprise')
]

_VALIDATION_DIRECTORIES_LABELS = [
  (_VALIDATION_ANGRY_FOLDER,'angry'),
  (_VALIDATION_DISGUST_FOLDER,'disgust'),
  (_VALIDATION_FEAR_FOLDER,'fear'),
  (_VALIDATION_HAPPY_FOLDER,'happy'),
  (_VALIDATION_NEUTRAL_FOLDER,'neutral'),
  (_VALIDATION_SAD_FOLDER,'sad'),
  (_VALIDATION_SURPRISE_FOLDER,'surprise')
]

In [15]:
import cv2
import os
import pandas as pd
import numpy as np
import random

_PICK_BATCH_SIZE = 5000
_CLASSIFICATION_SIZE = 7

def one_hot_encode(x,size=_CLASSIFICATION_SIZE):
  label = {'angry':0,'disgust':1,'fear':2,'happy':3,'neutral':4,'sad':5,'surprise':6}
  x = label[x]
  ohe = np.zeros(size)
  ohe[x]=1
  return ohe

training_images_labels_list_all = []
for training_directory_label in _TRAINING_DIRECTORIES_LABELS:
  label = training_directory_label[1]
  directory = training_directory_label[0]
  print(directory)
  for filename in os.listdir(directory):
      if filename.endswith('.jpg') or filename.endswith('.png') or filename.endswith('.jpeg'):
        if(len(training_images_labels_list_all)%100==0):
          print('.',end='')
        image = cv2.imread(os.path.join(directory, filename))
        training_images_labels_list_all.append([image,label])
      else:
          print(f'Unable to read image: {filename}')

print(f'Total images in the training set: {len(training_images_labels_list_all)}')

drive/MyDrive/A2_Data/T3_Data/images/train/angry
........................................drive/MyDrive/A2_Data/T3_Data/images/train/disgust
.....drive/MyDrive/A2_Data/T3_Data/images/train/fear
.........................................drive/MyDrive/A2_Data/T3_Data/images/train/happy
.......................................................................drive/MyDrive/A2_Data/T3_Data/images/train/neutral
..................................................drive/MyDrive/A2_Data/T3_Data/images/train/sad
..................................................drive/MyDrive/A2_Data/T3_Data/images/train/surprise
................................Total images in the training set: 28823


In [23]:
#test train split
def get_col(array,index):
  arr = []
  for row in array:
    arr.append(row[index])
  return arr

def encode_labels(y):
  arr = []
  for label in y:
    arr.append(one_hot_encode(label))
  return arr

from sklearn.model_selection import train_test_split
X = get_col(training_images_labels_list_all,0)
y = get_col(training_images_labels_list_all,1)
y = encode_labels(y)
X_train_imgs_locations, X_test_img_locations, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
len(X_train_imgs_locations), len(y_train), len(X_test_img_locations), len(y_test)

(23058, 23058, 5765, 5765)

In [19]:
# given a image_location, get that image
def get_img_from_location(location):
  if location.endswith('.jpg') or location.endswith('.png') or location.endswith('.jpeg'):
            # Read the image using OpenCV
            image = cv2.imread(location)
            # Append the image to the list and its name too
            #print(np.shape(image))
            if image is None:
                print(f'Unable to read image: {location}')
  return image

# given a list of images_locations, get those images
def get_imgs_from_locations(locations):
  images = []
  for location in locations:
    images.append(get_img_from_location(location))
  return images

In [20]:
#config model
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=14336)]) # Notice here
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


#declare model
history = []
model = None
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

_IMAGES_SHAPE = (48, 48, 3)
epochs = 20
batch_size = 10
model = Sequential([
    Conv2D(2, (2, 2), activation='relu', input_shape=_IMAGES_SHAPE),
    MaxPooling2D((2, 2)),
    Conv2D(4, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(8, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(16, (2, 2), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.6),
    Dense(7, activation='softmax')
])
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [26]:
#pick a number of images from the training set to train model on
train_images_list = X_train_imgs_locations
train_labels_list = y_train


In [33]:
# Split dataset
X_train, X_val, y_train, y_val = train_test_split(train_images_list, train_labels_list, test_size=0.2, random_state=42)
np.shape(X_train), np.shape(y_train), np.shape(X_val), np.shape(y_val)

((18446, 48, 48, 3), (18446, 7), (4612, 48, 48, 3), (4612, 7))

In [37]:


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.1
)
datagen.fit(X_train)
_IMAGES_SHAPE = (48, 48, 3)
# Model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=_IMAGES_SHAPE),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(_CLASSIFICATION_SIZE, activation='softmax')
])

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
print(np.shape(X_train), np.shape(y_train), np.shape(X_val), np.shape(y_val))
# Train the model
with tf.device('/device:GPU:0'):
  history = model.fit(np.array(X_train), np.array(y_train),  batch_size=32, validation_data=(np.array(X_val), np.array(y_val)), epochs=50)

(18446, 48, 48, 3) (18446, 7) (4612, 48, 48, 3) (4612, 7)
Epoch 1/50
577/577 [==============================] - 6s 7ms/step - loss: 3.2488 - accuracy: 0.2670 - val_loss: 1.7166 - val_accuracy: 0.3077
Epoch 2/50
577/577 [==============================] - 3s 6ms/step - loss: 1.6734 - accuracy: 0.3587 - val_loss: 1.8235 - val_accuracy: 0.2758
Epoch 3/50
577/577 [==============================] - 3s 6ms/step - loss: 1.5475 - accuracy: 0.4065 - val_loss: 1.5515 - val_accuracy: 0.4187
Epoch 4/50
577/577 [==============================] - 4s 7ms/step - loss: 1.4772 - accuracy: 0.4481 - val_loss: 1.5900 - val_accuracy: 0.4016
Epoch 5/50
577/577 [==============================] - 4s 6ms/step - loss: 1.3897 - accuracy: 0.4763 - val_loss: 1.5477 - val_accuracy: 0.4254
Epoch 6/50
577/577 [==============================] - 3s 5ms/step - loss: 1.3120 - accuracy: 0.5103 - val_loss: 1.6845 - val_accuracy: 0.4113
Epoch 7/50
577/577 [==============================] - 3s 5ms/step - loss: 1.2406 - accurac

In [38]:
from tensorflow.keras.models import save_model

# Define the file path where you want to save the model
model_path = 'facial_expression_recognission_1.0.h5'

# Save the model to the specified file path
model.save(model_path)

print(f"Model saved to {model_path}")

Model saved to facial_expression_recognission_1.0.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [40]:
print(history.history)

{'loss': [3.2488481998443604, 1.6733691692352295, 1.5474791526794434, 1.4772471189498901, 1.3896604776382446, 1.3119548559188843, 1.2405977249145508, 1.1606547832489014, 1.0917844772338867, 1.0394080877304077, 0.9721312522888184, 0.9070764183998108, 0.8389825224876404, 0.7834636569023132, 0.7372317910194397, 0.6850873231887817, 0.6260488629341125, 0.5818684697151184, 0.5394902229309082, 0.5209733843803406, 0.48346370458602905, 0.4673054814338684, 0.4407954514026642, 0.4309662878513336, 0.411111444234848, 0.3814040422439575, 0.39142948389053345, 0.35635247826576233, 0.3525505065917969, 0.35639098286628723, 0.344145804643631, 0.3285374045372009, 0.30775266885757446, 0.3219364583492279, 0.29480767250061035, 0.30188295245170593, 0.27067336440086365, 0.27073004841804504, 0.2549847662448883, 0.25330519676208496, 0.26421093940734863, 0.2494516372680664, 0.2735973000526428, 0.2616594135761261, 0.2382260113954544, 0.2336646020412445, 0.2538423240184784, 0.21386541426181793, 0.23775829374790192,